In [1]:
%load_ext autoreload
%autoreload 2

In [14]:
import ee
import geemap

PROJECT_ID = "ee-a01571000"

ee.Authenticate()
ee.Initialize(project=PROJECT_ID)

print(ee.String("Succesful connection to Google Earth Engine!").getInfo())  # type: ignore

Succesful connection to Google Earth Engine!


In [4]:
map = geemap.Map()
map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [4]:
# Dataset de imágenes obtenido de
# https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC08_C02_T1#description

img_set = ee.ImageCollection("LANDSAT/LC08/C02/T1")  # type: ignore
bands = img_set.select(["B4", "B3", "B5"])

# B3 - Green
# B4 - Red
# B5 - NIR (Near Infrared)

# Parece que normalmente la variable L (factor de ajuste del suelo) se pone como L = 0.5
# según en la página 4 https://catalogos.conae.gov.ar/landsat8/Docs/IndicesEspectralesDerivadosDeLandsat8.pdf

visualization = {
    min: 0.0,
    max: 30000.0,
}

# Actualizar el mapa interactivo
map.setCenter(-100.4, 25.73, 10)
map.addLayer(bands, visualization, "Green + Red + NIR")

In [5]:
# Define the Area of Interest (AOI)
# lat, long de esquina abajo izquiera y lat, long de esquina arriba derecha
# Se crea un triangulo con estos dos puntos, siendo la área

aoi = ee.Geometry.Rectangle(
    [-100.3543, 25.5664, -100.3049, 25.5989]
)  ### Aquí van las coordenadas para el área

In [9]:
# area metropolitana

aoi = ee.Geometry.Rectangle(
    [-100.432833, 25.616853, -100.124592, 25.838453]
)  ### Aquí van las coordenadas para el área

In [10]:
# Filter the Image Collection by AOI
filtered_images = img_set.filterBounds(aoi)

In [11]:
# Count the number of images in the filtered collection
image_count = filtered_images.size().getInfo()
print(f"Number of images available: {image_count}")

Number of images available: 250


In [12]:
# Get the oldest and newest images based on the 'system:time_start' property
oldest_image = filtered_images.sort("system:time_start").first()
newest_image = filtered_images.sort("system:time_start", False).first()

# Get the dates of the oldest and newest images
oldest_date = (
    ee.Date(oldest_image.get("system:time_start")).format("YYYY-MM-dd").getInfo()
)
newest_date = (
    ee.Date(newest_image.get("system:time_start")).format("YYYY-MM-dd").getInfo()
)

print(f"Oldest image date: {oldest_date}")
print(f"Newest image date: {newest_date}")

Oldest image date: 2013-03-30
Newest image date: 2024-08-04


In [15]:
# sierra madre

aoi = ee.Geometry.Rectangle(
    [-100.51304877876831, 28.708545044994917, -100.62115423654488, 25.009350725717105]
)  ### Aquí van las coordenadas para el área

In [16]:
# Filter the Image Collection by AOI
filtered_images = img_set.filterBounds(aoi)

# Count the number of images in the filtered collection
image_count = filtered_images.size().getInfo()
print(f"Number of images available: {image_count}")

# Get the oldest and newest images based on the 'system:time_start' property
oldest_image = filtered_images.sort("system:time_start").first()
newest_image = filtered_images.sort("system:time_start", False).first()

# Get the dates of the oldest and newest images
oldest_date = (
    ee.Date(oldest_image.get("system:time_start")).format("YYYY-MM-dd").getInfo()
)
newest_date = (
    ee.Date(newest_image.get("system:time_start")).format("YYYY-MM-dd").getInfo()
)

print(f"Oldest image date: {oldest_date}")
print(f"Newest image date: {newest_date}")

Number of images available: 1452
Oldest image date: 2013-03-30
Newest image date: 2024-08-04


---


In [22]:
import ee
import geemap

# Initialize the Earth Engine module
ee.Initialize()

# 1. Define the metropolitan area of Monterrey as an irregular polygon
municipiosNL = ee.Geometry.Polygon(
    [
        [
            [-100.4645, 25.9021],  # Apodaca
            [-100.2631, 25.8704],  # Guadalupe
            [-100.2122, 25.7810],  # Juárez
            [-100.2510, 25.6191],  # Santiago
            [-100.4528, 25.5653],  # Santa Catarina
            [-100.5640, 25.6294],  # García
            [-100.5806, 25.8335],  # García
            [-100.5177, 25.9407],  # Escobedo
            [-100.4645, 25.9021],  # Closing the polygon
        ]
    ]
)

# Create a map
Map = geemap.Map(center=[25.6715, -100.309], zoom=10)

# Add the metropolitan area polygon to the map
Map.addLayer(municipiosNL, {}, "Monterrey Metropolitan Area")
Map.centerObject(municipiosNL)

points = ee.FeatureCollection.randomPoints(
    region=municipiosNL,
    points=100_000,
    seed=0,
    maxError=1,
)


Map.addLayer(points, {"color": "red"}, "Random Points")

# 3. Daily NDVI and NDWI Summary for Monterrey Metropolitan Area (2015-2016)
# NDVI Collection
modisNDVI = (
    ee.ImageCollection("MODIS/MOD09GA_NDVI")
    .select("NDVI")
    .filterDate("2015-01-01", "2016-12-31")
    .map(
        lambda img: img.set(
            {
                "year": ee.Date(img.get("system:time_start")).get("year"),
                "month": ee.Date(img.get("system:time_start")).get("month"),
                "day": ee.Date(img.get("system:time_start")).get("day"),
                "date": ee.Date(img.get("system:time_start")),
            }
        )
    )
)

# NDWI Collection
modisNDWI = (
    ee.ImageCollection("MODIS/MOD09GA_006_NDWI")
    .select("NDWI")
    .filterDate("2015-01-01", "2016-12-31")
    .map(
        lambda img: img.set(
            {
                "year": ee.Date(img.get("system:time_start")).get("year"),
                "month": ee.Date(img.get("system:time_start")).get("month"),
                "day": ee.Date(img.get("system:time_start")).get("day"),
                "date": ee.Date(img.get("system:time_start")),
            }
        )
    )
)


# Combine NDVI and NDWI values for each point
def combine_indices(ndviImg):
    correspondingNdwiImg = modisNDWI.filter(
        ee.Filter.eq("date", ndviImg.get("date"))
    ).first()
    return points.map(
        lambda point: ee.Feature(
            point.geometry(),
            {
                "date": ndviImg.get("date"),
                "year": ndviImg.get("year"),
                "month": ndviImg.get("month"),
                "day": ndviImg.get("day"),
                "NDVI": ndviImg.reduceRegion(
                    reducer=ee.Reducer.mean(),
                    geometry=point.geometry(),
                    scale=500,
                    bestEffort=True,
                    maxPixels=1e8,
                ).get("NDVI"),
                "NDWI": correspondingNdwiImg.reduceRegion(
                    reducer=ee.Reducer.mean(),
                    geometry=point.geometry(),
                    scale=500,
                    bestEffort=True,
                    maxPixels=1e8,
                ).get("NDWI"),
            },
        )
    )


combinedIndices = (
    modisNDVI.map(combine_indices).flatten().filter(ee.Filter.notNull(["NDVI", "NDWI"]))
)

# 4. Export multiple daily NDVI and NDWI values as CSV file format
# task = ee.batch.Export.table.toDrive(
#     collection=combinedIndices,
#     description="NDVI_NDWI_Metropolitana_2015_2016_MultiPoint",
#     fileFormat="CSV",
# )
# task.start()

# Display the map1
Map

Map(center=[25.6715, -100.309], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchD…

In [16]:
import google
import google.oauth2.credentials
from google.auth import compute_engine
import google.auth.transport.requests


def idtoken_from_metadata_server(url: str):
    """
    Use the Google Cloud metadata server in the Cloud Run (or AppEngine or Kubernetes etc.,)
    environment to create an identity token and add it to the HTTP request as part of an
    Authorization header.

    Args:
        url: The url or target audience to obtain the ID token for.
            Examples: http://www.example.com
    """

    request = google.auth.transport.requests.Request()
    # Set the target audience.
    # Setting "use_metadata_identity_endpoint" to "True" will make the request use the default application
    # credentials. Optionally, you can also specify a specific service account to use by mentioning
    # the service_account_email.
    credentials = compute_engine.IDTokenCredentials(
        request=request, target_audience=url, use_metadata_identity_endpoint=True
    )

    # Get the ID token.
    # Once you've obtained the ID token, use it to make an authenticated call
    # to the target audience.
    credentials.refresh(request)
    # print(credentials.token)
    print("Generated ID token.")